In [1]:
import pandas as pd
import requests
import time
import numpy as np
import glob
import bs4
import pypyodbc as podbc
import pygsheets
import gspread

In [7]:
this_year = '2020-21'

#  Update totals

In [3]:
#Function to scrape per-game values
def scrape_totals(season):
    headers = {
'Host': 'stats.nba.com',
'Connection': 'keep-alive',
'Accept': 'application/json, text/plain, */*',
'x-nba-stats-token': 'true',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
'x-nba-stats-origin': 'stats',
'Referer': 'https://www.nba.com/',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9'}

    url = f"https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season={season}&SeasonType=Regular+Season&StatCategory=PTS"
    r = requests.get(url, headers=headers).json()

    df = pd.DataFrame(r['resultSet']['rowSet'], columns = r['resultSet']['headers'])
    df['Season'] = season
    return df

In [ ]:
thisyear = scrape_totals(this_year)
pastyears = pd.read_csv(r'C:\\Users\gsteele\Other\totals.csv')
pastyears = pastyears[pastyears['Season'] != this_year]
frames = [thisyear,pastyears]
season_totals = pd.concat(frames)

In [ ]:
#Calculate two-pointers
season_totals.insert(loc = 28, column = 'FG2M', value = (season_totals['FGM'] - season_totals['FG3M']))
season_totals.insert(loc = 28, column = 'FG2A', value = (season_totals['FGA'] - season_totals['FG3A']))

#DraftKings average
season_totals.insert(loc = 1, column = 'draftkings', value = (
        (season_totals.FG3M*3.5)+(season_totals.FG2M*2)+(season_totals.FTM)+(season_totals.REB*1.25)+
        (season_totals.BLK*2)+(season_totals.STL*2)+(season_totals.TOV*(-0.5))+(season_totals.AST*1.5)
        )
)
#FanDuel average
season_totals.insert(loc = 1, column = 'fanduel', value = (
        (season_totals.FG3M*3)+(season_totals.FG2M*2)+(season_totals.FTM)+(season_totals.REB*1.2)+
        (season_totals.BLK*2)+(season_totals.STL*2)+(season_totals.TOV*(-1))+(season_totals.AST*1.5)
        )
)

season_totals = season_totals[['PLAYER_ID','PLAYER','Season','fanduel','draftkings','TEAM','GP','MIN','PTS','FGM',
                     'FGA','FG2M','FG2A','FG3M','FG3A','FG_PCT','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','TOV',
                    'STL','BLK','PF']]

In [ ]:
season_totals['FG3M'] = season_totals['FG3M'].fillna(value = 0)

season_totals.insert(loc = 28, column = 'eFG', value = (np.where(season_totals['FGA'] == 0, 0,
                            (((season_totals.FGM + (0.5*season_totals.FG3M))
                            /season_totals.FGA)))))

season_totals.insert(loc = 28, column = 'TS', value = (np.where((season_totals['FGA'] == 0) & (season_totals['FTA'] == 0), 0,
                            (season_totals.PTS / 
                               (2*(season_totals.FGA + (0.44*season_totals.FTA)))))))

season_totals = season_totals.fillna(value = 0)

In [ ]:
spread_client = gspread.service_account(filename='C:\\Users\gsteele\Other\loyal-skill-297317-d353a7b193db.json')
sheet = spread_client.open('Totals')
worksheet = sheet.worksheet("Totals")
worksheet.update([season_totals.columns.values.tolist()] + season_totals.values.tolist())

#  Update pace

In [16]:
#Connect to RDS db
conn = podbc.connect(
    #Trusted_Connection='Yes',
    Driver='{SQL Server}',
    Server='nbahistorical.cmgzaupuq9dz.us-east-2.rds.amazonaws.com',
    UID='admin',
    PWD='Gs239350',
    Database='NBA_historical',
    MultipleActiveResultSets = 'True'
)

In [17]:
df = pd.read_sql_query(sql = """
select *
from (
select
    season_id,
    team,
    team_id,
    (
        (
        sum(game_poss_per_min) over (partition by team_id,season_id order by team_id)
        / 
        count(game_id) over(partition by team_id,season_id order by team_id)
        )
        *48
    ) as pace,
    row_number() over (partition by team_id,season_id order by team_id) as r
from game_pace
) as subquery
where r = 1;
""", con = conn)

In [18]:
#Function to scrape current season pace values from nba.com
def scrape_pace(season):
    headers = {
'Host': 'stats.nba.com',
'Connection': 'keep-alive',
'Accept': 'application/json, text/plain, */*',
'x-nba-stats-token': 'true',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
'x-nba-stats-origin': 'stats',
'Referer': 'https://www.nba.com/',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9'}

    url = f"https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision="
    r = requests.get(url, headers=headers).json()

    df = pd.DataFrame(r['resultSets'][0]['rowSet'], columns = r['resultSets'][0]['headers'])
    df['Season'] = season
    return df

In [32]:
current = scrape_pace(this_year)
current = current[['Season','TEAM_ID','PACE']]
abb = df[['team','team_id']].drop_duplicates()
current = current.merge(right = abb, how = 'left', left_on = 'TEAM_ID', right_on = 'team_id')
current = current[['Season','team', 'TEAM_ID','PACE']]
current.insert(loc = 4, column = 'r', value = 1)
current = current.rename(columns = {"Season":"season_id","TEAM_ID":"team_id","PACE":"pace"})
all_pace_values = [df,current]
pace = pd.concat(all_pace_values)

In [35]:
spread_client = gspread.service_account(filename='C:\\Users\gsteele\Other\loyal-skill-297317-d353a7b193db.json')
sheet = spread_client.open('Totals')
worksheet = sheet.worksheet("Pace")
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1gpdS21XBTwCVeC0nih1f_-5htFV6_qztoat5_SuYCVM',
 'updatedRange': 'Pace!A1:E1533',
 'updatedRows': 1533,
 'updatedColumns': 5,
 'updatedCells': 7665}

In [ ]:
#worksheet.update([season_totals.columns.values.tolist()] + season_totals.values.tolist())

In [ ]:
#years = ['2019-20','2018-19','2017-18','2016-17','2015-16','2014-15','2013-14','2012-13','2011-12','2010-11','2009-10','2008-09','2007-08',
#        '2006-07','2005-06','2004-05','2003-04','2002-03','2001-02','2000-01','1999-00','1998-99','1997-98','1996-97','1995-96',
#        '1994-95','1993-94','1992-93','1991-92','1990-91','1989-90','1988-89','1987-88','1986-87','1985-86','1984-85','1983-84',
#        '1982-83','1981-82','1980-81','1979-80','1978-79','1977-78','1976-77','1975-76','1974-75','1973-74',
#        '1972-73','1971-72','1970-71','1969-70','1968-69','1967-68','1966-67','1965-66','1964-65','1963-64','1962-63','1961-62',
#        '1960-61','1959-60','1958-59','1957-58','1956-57','1955-56','1954-55','1953-54','1952-53','1951-52','1950-51',
#          '1949-50','1948-49','1947-48','1946-47']

In [ ]:
#Combine relevant per-game values from past seasons
#frame_list=[]
#for season in years:
#    time.sleep(np.random.randint(0,5 + 1))
#    df = scrape_totals(season)
#    frame_list.append(df)
#
#season_totals = pd.concat(frame_list, ignore_index = True)
#season_totals.to_csv("C:\\Users\gsteele\Other\\totals.csv")

In [ ]:
#season_totals = pd.read_csv('C:\\Users\gsteele\other\season_totals.csv')

In [ ]:
#eff = pd.read_sql(sql = """
#set arithabort off set ansi_warnings off;
#select * from offensive_efficiency
#where PLAYER = 'DeMarcus Cousins';
#""", con = conn)
#
#eff